# Indexing MDFs with Pony ORM.


"SDK"/Guide to developing your own MDF management & analysis solution. 

This can scale from one project with a bunch of log files sitting on your laptop to a full corporate S3 data-lake.

In [1]:
import fsspec
from asammdf import MDF

In [1]:
import fsspec
import os
s3_cfg = {
    "key": "mdf_minio_access_key",
    "secret": "mdf_minio_secret_key",
    "client_kwargs": {
        "endpoint_url": "http://minio:9000",
    },
}
fs = fsspec.filesystem("s3", **s3_cfg)
buckets = fs.ls("")
print(f"Buckets: {fs.ls('')}")

Buckets: ['mdfbucket-0', 'mdfbucket-1', 'mdfbucket-2', 'mdfbucket-3', 'mdfbucket-4', 'mdfbucket-5', 'mdfbucket-6', 'mdfbucket-7', 'mdfbucket-8', 'mdfbucket-9', 'test']


# Walk Through All Files:

Walk through all S3 files and find the first one.

In [2]:
import os
mdf_paths=list()

fs = fsspec.filesystem("s3", **s3_cfg)
for bucket in fs.ls(""):
    for root, dirs, files in fs.walk(bucket):
        for file in files:
            if file.lower().endswith(".mf4") or file.lower().endswith(".mdf"):
                mdf_paths.append(os.path.join(root, file))

Randomly pick a file for analysis/postprocessing.

In [4]:
import random
mdf_path = random.choice(mdf_paths)
fs.info(mdf_path)

{'Key': 'mdfbucket-4/DäsCarGmbh/Excavator/6102bee5-8ec7-4a95-b695-9e6d5d837e7f.mf4',
 'LastModified': datetime.datetime(2020, 4, 21, 20, 17, 30, 104000, tzinfo=tzlocal()),
 'ETag': '"74810c962909f2cae91bf7e00b83ae5e"',
 'Size': 603648,
 'StorageClass': 'STANDARD',
 'Owner': {'DisplayName': '', 'ID': ''},
 'type': 'file',
 'size': 603648,
 'name': 'mdfbucket-4/DäsCarGmbh/Excavator/6102bee5-8ec7-4a95-b695-9e6d5d837e7f.mf4'}

Determine what keys we need to make:

In [5]:
for k,v in fs.info(mdf_path).items():
    print(f"{k}{type(v)}")

Key<class 'str'>
LastModified<class 'datetime.datetime'>
ETag<class 'str'>
Size<class 'int'>
StorageClass<class 'str'>
Owner<class 'dict'>
type<class 'str'>
size<class 'int'>
name<class 'str'>


Minimal skeleton Pony classes. 

These can be adapted to suit your needs. For example the MDF class could contain the vehicle S/N or other information.

In [6]:
import os

import pony.orm
from pony.orm.core import EntityMeta
from datetime import datetime

pony.orm.set_sql_debug(False)
db = pony.orm.Database()

if True:
    # Inmemory datatabase
    filename=":memory:"
else:
    # Or not.
    filename = os.path.abspath("mdf_index.sqlite")
    if os.path.exists(filename):
        os.unlink(filename)
# Bind
db.bind(
    provider="sqlite", filename=filename, create_db=True,
)

#For S3 Indexing
class MDF(db.Entity):
    # fs
    key = pony.orm.Required(str, unique=True,)
    last_modified = pony.orm.Optional(datetime,)
    etag = pony.orm.Optional(str,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    storage_class = pony.orm.Optional(str,)
    type = pony.orm.Optional(str,)
    name = pony.orm.Optional(str,)
    # Basename.
    basename = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
    
class Channel(db.Entity):
    """Channel entity to represent a 
    
    """
    name = pony.orm.Required(str, unique=True,)
    mdfs = pony.orm.Set("MDF",)


db.generate_mapping(create_tables=True)

def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: Additional fields to set if ```get``` returns nothing.
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), f"{cls} is not a database entity"

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        obj = cls(**set, **get)
        db.commit()
        return obj
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj

In [7]:
info = fs.info(mdf_path)

In [8]:
info

{'Key': 'mdfbucket-4/DäsCarGmbh/Excavator/6102bee5-8ec7-4a95-b695-9e6d5d837e7f.mf4',
 'LastModified': datetime.datetime(2020, 4, 21, 20, 17, 30, 104000, tzinfo=tzlocal()),
 'ETag': '"74810c962909f2cae91bf7e00b83ae5e"',
 'Size': 603648,
 'StorageClass': 'STANDARD',
 'Owner': {'DisplayName': '', 'ID': ''},
 'type': 'file',
 'size': 603648,
 'name': 'mdfbucket-4/DäsCarGmbh/Excavator/6102bee5-8ec7-4a95-b695-9e6d5d837e7f.mf4'}

In [9]:
def index_mdf(mdf_path):
    info = fs.info(mdf_path)
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"key": info["Key"]},
    set={
        "last_modified": info["LastModified"],
        "etag": info["ETag"],
        "size": info["size"],
        "size_mb": info["size"] / 1024 ** 2,
        "storage_class": info["StorageClass"],
        "type": info["type"],
        "name": info["name"],
        "basename": os.path.basename(info["name"])
        },
    )
    try:
        db.commit()
        return MDF_
    except:
        db.rollback()
        return None

In [10]:
print(f"Indexing {len(mdf_paths)} MDF files")

Indexing 975 MDF files


Benchmark indexing times:

In [11]:
%%timeit
for idx, mdf_path in enumerate(mdf_paths):
    index_mdf(mdf_path)

441 ms ± 2.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Index one file and get the orm object:

In [12]:
mdf = index_mdf(mdf_path)
mdf

MDF[435]

In [13]:
# Add features to the class.
def __repr__(self):
    return f"MDF<{self.basename}>"
setattr(MDF, "__repr__", __repr__)
mdf

MDF<6102bee5-8ec7-4a95-b695-9e6d5d837e7f.mf4>

# Adding Other Attributes

[To adapt to your own needs].

In the data example the company name & product are embedded in the mdf file path, we can extract them out to add to the database.

In [17]:
os.path.basename(os.path.dirname(mdf.name))

'Excavator'

In [18]:
os.path.basename(
    os.path.dirname(
        os.path.dirname(
            mdf.name
        )
    )
)

'DäsCarGmbh'

In [19]:
product = os.path.basename(os.path.dirname(mdf.name))
company = os.path.basename(
    os.path.dirname(
        os.path.dirname(
            mdf.name
        )
    )
)

Create property functions to return the product & company given a filepath:

In [20]:
def product(self):
    return os.path.basename(os.path.dirname(self.name))
def company(self):
    return os.path.basename(
        os.path.dirname(
            os.path.dirname(
                mdf.name
            )
        )
    )

Doesn't exist:

In [21]:
mdf.product

AttributeError: 'MDF' object has no attribute 'product'

Attach the properties to the MDF class:

In [22]:
setattr(MDF, 'product', property(product))
setattr(MDF, 'company', property(company))

In [23]:
mdf.product

'Excavator'

In [24]:
mdf.company

'DäsCarGmbh'

In [25]:
db.rollback()
mdfs = pony.orm.select(m for m in MDF)

for mdf in mdfs:
    break
mdf

MDF<085de752-222e-44ce-a1d9-53c6352cd038.mf4>

# MDF Analysis.

The above analysis was based on just the file attributes and attributes embedded in the filename (Company, Product).

This step uses ```asammdf``` to index MDF attributes such as which channels each file contains.

In [29]:
import asammdf

In [30]:
with fs.open(mdf.name) as fid:
    mdf_ = asammdf.MDF(fid)

In [31]:
mdf_.channels_db

{'time': ((0, 0),),
 'engine_speed': ((0, 1),),
 'engine_speed_desired': ((0, 2),),
 'vehicle_speed': ((0, 3),),
 'coolant_temp': ((0, 4),),
 'longitude': ((0, 5),),
 'latitude': ((0, 6),),
 'power': ((0, 7),),
 'efficiency': ((0, 8),),
 'ADAS5_failure': ((0, 9),),
 'X': ((0, 10),),
 'Y': ((0, 11),),
 'Z': ((0, 12),)}

In [32]:
channels=list()
for channel in mdf_.channels_db.keys():
    channel_ = upsert(Channel, {"name": channel})
    channels.append(channel_)

In [33]:
channels

[Channel[1],
 Channel[2],
 Channel[3],
 Channel[4],
 Channel[5],
 Channel[6],
 Channel[7],
 Channel[8],
 Channel[9],
 Channel[10],
 Channel[11],
 Channel[12],
 Channel[13]]

In [34]:
mdf.key

'mdfbucket-0/ABMøøse/BoatyMcBoatface/085de752-222e-44ce-a1d9-53c6352cd038.mf4'

In [46]:
def process_channels(mdfs):
    """Given a list of MDF files, process the channels
    
    """
    for mdf in mdfs:
        # Open the MDF file.
        try:
            with fs.open(mdf.name, "rb") as fid:
                mdf_ = asammdf.MDF(fid)
        except asammdf.mdf.MdfException:
            # Bad file!
            continue
        # 
        channels=list()
        # Loop through each of the channels in the database.
        for channel in mdf_.channels_db.keys():
            channel_ = upsert(Channel, {"name": channel})
            channels.append(channel_)
        MDF_ = upsert(
        cls=MDF,
        get={"key": mdf.name},
        set={
            "channels": channels
            },
        )
        db.commit()

In [47]:
mdf

MDF<085de752-222e-44ce-a1d9-53c6352cd038.mf4>

In [48]:
process_channels([mdf])

In [49]:
%%timeit
# How long does it take to insert channels for 10 mdfs
process_channels(mdfs[:10])

182 ms ± 6.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
# Insert channel information for each of the MDFs
process_channels(mdfs)

In [50]:
channels = pony.orm.select(m for m in Channel)

In [51]:
for channel in channels:
    break

In [52]:
len(list(channel.mdfs))

975

How many MDFs contain each channel?

In [53]:
for channel in channels:
    n = len(list(channel.mdfs))
    print(f"{channel.name}: {n}")

time: 975
engine_speed: 967
engine_speed_desired: 964
vehicle_speed: 969
coolant_temp: 965
longitude: 959
latitude: 969
power: 964
efficiency: 965
ADAS5_failure: 967
X: 959
Y: 966
Z: 962


# Further Development.

Take the minimal skeleton from above and develop it into a more 'useful' example with some additional python.

In [68]:
import os

import pony.orm
from pony.orm.core import EntityMeta
from datetime import datetime

pony.orm.set_sql_debug(False)
db = pony.orm.Database()

# Work in memory, or not.
if True:
    # Inmemory datatabase
    filename=":memory:"
else:
    # Or not.
    filename = os.path.abspath("mdf_index.sqlite")
    if os.path.exists(filename):
        os.unlink(filename)
# Bind
db.bind(
    provider="sqlite", filename=filename, create_db=True,
)


# For Local Indexing.
class MDF(db.Entity):
    # Filesystem Bits.
    key = pony.orm.Required(str, unique=True,)
    last_modified = pony.orm.Optional(datetime,)
    etag = pony.orm.Optional(str,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    storage_class = pony.orm.Optional(str,)
    type = pony.orm.Optional(str,)
    name = pony.orm.Optional(str,)
    
    # Pre-calculated bits.
    basename = pony.orm.Optional(str,)
    
    product = pony.orm.Optional(str,)
    company = pony.orm.Optional(str,)

    # ASAM MDF Bits.
    version = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
    
    def __repr__(self):
        return f"MDF<{self.id},{self.product},{self.company},Ch:{len(self.channels)}>"

class Channel(db.Entity):
    """Channel entity to represent a 
    
    """
    name = pony.orm.Required(str, unique=True,)
    mdfs = pony.orm.Set("MDF",)
    
    def __repr__(self):
        return f"Channel<{self.id},{self.name}>"

def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: Additional fields to set if ```get``` returns nothing.
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), f"{cls} is not a database entity"

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj
    

db.generate_mapping(create_tables=True)

def index_mdf(mdf_path):
    info = fs.info(mdf_path)
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"key": info["Key"]},
    set={
        "last_modified": info["LastModified"],
        "etag": info["ETag"],
        "size": info["size"],
        "size_mb": info["size"] / 1024 ** 2,
        "storage_class": info["StorageClass"],
        "type": info["type"],
        "name": info["name"],
        "basename": os.path.basename(info["name"])
        },
    )
    try:
        db.commit()
        return MDF_
    except:
        db.rollback()
        return None
import asammdf
def index_channels(mdf):
    """Given a MDF files, process the channels
    
    """
    # Open the MDF file.
    mdf_ = asammdf.MDF(mdf.name)
    # 
    channels=list()
    # Loop through each of the channels in the database.
    for channel in mdf_.channels_db.keys():
        channel_ = upsert(Channel, {"name": channel})
        channels.append(channel_)
    MDF_ = upsert(
    cls=MDF,
    get={"name": mdf.name},
    set={
        "channels": channels
        },
    )
    try:
        db.commit()
        return channels
    except:
        db.rollback()
        return None
        
def index_mdf_info(mdf):
    """ Index company and product information in the database from the filename."""
    product = os.path.basename(os.path.dirname(mdf.name))
    company = os.path.basename(
        os.path.dirname(
            os.path.dirname(
                mdf.name
            )
        )
    )
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"name": mdf.name},
    set={
        "product": product,
        "company": company,
        },
    )
    try:
        db.commit()
        return MDF_
    except:
        db.rollback()
        return None

In [63]:
import os
mdf_paths=list()

fs = fsspec.filesystem("s3", **s3_cfg)
for bucket in fs.ls(""):
    for root, dirs, files in fs.walk(bucket):
        for file in files:
            if file.lower().endswith(".mf4") or file.lower().endswith(".mdf"):
                mdf_paths.append(os.path.join(root, file))
print(f"Found {len(mdf_paths)} MDF files")

Found 975 MDF files


Randomly pick a file for analysis.

In [66]:
mdf_path = random.choice(mdf_paths)
mdf_path

'mdfbucket-8/재벌/Car/eee3eac1-693a-4b8b-ac6f-b708a33b7e4a.mf4'

Insert the MDF file into the database.

[Notice the __repr__ string isn't fully populated, the data isn't yet in the database]

In [67]:
mdf = index_mdf(mdf_path)
mdf

MDF<1,,,Ch:0>

Index the product and company name of the mdf

In [228]:
index_mdf_info(mdf)
mdf

MDF<1,Car,DanishStartup,Ch:0>

Index the channels.

In [229]:
index_channels(mdf)
mdf

.............


MDF<1,Car,DanishStartup,Ch:13>

In [230]:
%%timeit
for mdf_path in mdf_paths[:10]:
    index_mdf(mdf_path)

9.66 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [105]:
pony.orm.select(m for m in MDF)[0:10]

[MDF[1], MDF[2], MDF[3], MDF[4], MDF[5], MDF[6], MDF[7], MDF[8], MDF[9], MDF[10]]

In [232]:
%%timeit
for mdf in pony.orm.select(m for m in MDF)[0:10]:
    index_mdf_info(mdf)

8.03 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [233]:
pony.orm.select(m for m in MDF)[0:10]

[MDF<1,Car,DanishStartup,Ch:13>, MDF<2,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:0>, MDF<3,Excavator,CarCompanyGmbh,Ch:0>, MDF<4,DumpTruck,FAANG-Spinof,Ch:0>, MDF<5,Transmission,FAANG-Spinof,Ch:0>, MDF<6,Transmission,FAANG-Spinof,Ch:0>, MDF<7,Car,FAANG-Spinof,Ch:0>, MDF<8,Bulldozer,FAANG-Spinof,Ch:0>, MDF<9,Bulldozer,FAANG-Spinof,Ch:0>, MDF<10,BoatyMcBoatfaceMarsColonizer,FAANG-Spinof,Ch:0>]

In [234]:
%%timeit
for mdf in pony.orm.select(m for m in MDF)[0:10]:
    index_channels(mdf)

.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
......

.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
......

This task can easily be distributed with celery or rq

[Asynchronous Task Execution In Python](https://bhavaniravi.com/blog/asynchronous-task-execution-in-python/)

In [235]:
for mdf_path in mdf_paths:
    mdf = index_mdf(mdf_path)
    index_mdf_info(mdf)
    index_channels(mdf)

.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............

.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
............
............
.............
.............
............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.

...

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
...........
.............
.............
.............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
...........
.............
.............
............

In [239]:
channels = pony.orm.select(c for c in Channel)

In [240]:
for channel in channels:
    break

How many MDF files have been indexed?

In [243]:
len(channel.mdfs)

6823

How many bytes of MDF files have been indexed?

In [244]:
pony.orm.sum(m.size for m in MDF)

16447456296

How many GB of MDF files have been indexed?

In [247]:
pony.orm.sum(m.size for m in MDF)/1024**3

15.317887343466282

Find the biggest MDF file to analyze:

In [252]:
q = pony.orm.select(mdf for mdf in MDF).order_by(lambda: pony.orm.desc(mdf.size))

In [253]:
q[0:5]

[MDF<40,Car,DanishStartup,Ch:13>, MDF<49,Car,DanishStartup,Ch:13>, MDF<70,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>, MDF<74,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>, MDF<80,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>]

In [255]:
mdf_ = asammdf.MDF(list(q)[0].name)

In [256]:
for chan in mdf_.iter_channels():
    chan.plot()

/projects/Jupyter_MDF_Analysis2/venv3.8/lib/python3.8/site-packages/matplotlib/axes/_base.py:2447: RuntimeWarning: overflow encountered in double_scalars
  delta = (x1t - x0t) * margin
/projects/Jupyter_MDF_Analysis2/venv3.8/lib/python3.8/site-packages/matplotlib/ticker.py:2198: RuntimeWarning: invalid value encountered in greater_equal
  istep = np.nonzero(steps >= raw_step)[0][0]


IndexError: index 0 is out of bounds for axis 0 with size 0